# Scraping Data from LinkedIn
   ### Things to scrape:
1. Major/title
2. Bio/description
3. Experience
4. Skills
5. Education


In [1]:
#User Name
allLinks = []
userName = []
imageLinks = []
biographyUser = []
# Recent Education Lists
schooling, degree_name, field_of_study, period_attended, gpa, activities = [],[],[],[],[],[]
#to be removed internally (DON'T REMOVE IT THO!!!)
education = []
#Skills of a User
skills = []
#Interests of a User
user_interests = []

In [2]:
from selenium import webdriver
import pandas as pd
import os
import re
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
br = webdriver.Chrome(executable_path=r"C:/Users/qasim/Downloads/chromedriver_win32 Official/chromedriver.exe")
br.get("https://www.linkedin.com/uas/login?trk=guest_homepage-basic_nav-header-signin")
time.sleep(2)
login_email = br.find_element_by_id("username")
login_email.send_keys("qasim31wani@gmail.com")
login_password = br.find_element_by_id("password")
login_password.send_keys("quinnwani")
login_password.submit()
time.sleep(0.32)
br.get("https://www.linkedin.com/search/results/people/?origin=DISCOVER_FROM_SEARCH_HOME")
time.sleep(0.49)
allFilters = ".search-filters-bar__all-filters.flex-shrink-zero.mr3.artdeco-button.artdeco-button--muted.artdeco-button--2.artdeco-button--tertiary.ember-view"
br.find_element_by_css_selector(allFilters).click()
time.sleep(0.2)
br.find_element_by_css_selector("#search-advanced-school").send_keys("Virginia Tech")
br.find_element_by_css_selector(".search-advanced-facets__button--apply.ml4.mr2.artdeco-button.artdeco-button--3.artdeco-button--primary.ember-view").click()

In [ ]:
def start():
    for slow in range(30):
        time.sleep(0.2)
        br.execute_script("window.scrollTo(0,"+ str(slow*100) +");")
start()

In [ ]:
def isElement(x):
    try:
        br.find_element_by_css_selector(x)
    except NoSuchElementException:
        return False
    return True

In [ ]:
def findDetailedInformation():
#   Image Link URL Storage
    time.sleep(3)
    if(isElement(".profile-photo-edit__preview") == True):
        imageLength = br.find_element_by_css_selector(".profile-photo-edit__preview").get_attribute('style').split("(\"")
    else:
        im = br.find_element_by_css_selector(".pv-top-card-section__photo.presence-entity__image.EntityPhoto-circle-9.ember-view")
        imageLength = im.get_attribute('style').split("(\"")
    if(len(imageLength)>1):
        newIm = im.get_attribute('style').split("(\"")[1]
        imageLinks.append(newIm[:-3])
    else:
        imageLinks.append("http://ichef.bbci.co.uk/news/999/mcs/media/images/80888000/jpg/_80888657_anon.jpg")
    
#  Biography:
    biography()

# Users' educational Details :
    individual_user_education()
    
# Users' Skills :
    userSkills()
    
# User Interests:
    br.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    user_has_interests()

In [ ]:
def biography():
    isMoreContent = ".pv-top-card-section__summary-toggle-button.pv-profile-section__card-action-bar.artdeco-container-card-action-bar.mt4"
    loadBio = ".pv-top-card-section__summary-text.mt4.ember-view"
    bxb = "No Biography"
    moreContentExists = isElement(isMoreContent)
    if(moreContentExists == True):
         br.find_element_by_css_selector(isMoreContent).click()
    if(isElement(loadBio) == True):
        bxb = br.find_element_by_css_selector(loadBio).text
    biographyUser.append(bxb)

In [ ]:
def individualProfiles():
    pN = ".search-result__result-link.ember-view"
    start()
    br.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(0.4)
    profileNames = br.find_elements_by_css_selector(pN)
    time.sleep(0.49)
    br.execute_script("window.scrollTo(document.body.scrollHeight,0);")
    even_flag = 0
    initial_len = len(allLinks)
    for x in profileNames:
        if(even_flag % 2 != 0):
            if(x.text.split("\n")[0] != "LinkedIn Member"):
                allLinks.append(x.get_attribute('href'))
                userName.append(x.text.split("\n")[0])
        even_flag += 1
    for x in range(initial_len,len(allLinks)):
        br.switch_to.window(br.window_handles[-1])
        br.get(allLinks[x])
        findDetailedInformation()
    br.switch_to.window(br.window_handles[0])

In [ ]:
def educationDetails(listName, x, startIndex):
    if(isListElementExist(listName, x, startIndex) == True):
        return listName.index(x, startIndex)
    return -9

def isListElementExist(listName, x, startIndex = 0):
    try:
        listName.index(x, startIndex)
    except ValueError:
        return False
    return True


In [ ]:
def individual_user_education():
    for x in range(40):
        br.execute_script("window.scrollTo(0,"+ str(x*100) +");")
        time.sleep(0.1)
    br.execute_script("window.scrollTo("+str(4000)+", document.body.scrollHeight);")
    time.sleep(0.2)
    i = br.find_element_by_id("education-section")
    education = i.text.split("\n")
    schooling.append(education[1])
    try:  
        dn = educationDetails(education,"Degree Name", 0)
        if(dn == -9):
            degree_name.append("NO DEGREE LISTED")
        else:
            degree_name.append(education[dn + 1])
    except TypeError:  
            print ("Type error raised in individual_user_education()" )

    fos = educationDetails(education, "Field Of Study", 0)
    if(fos == -9):
        field_of_study.append("NO FIELD OF STUDY LISTED")
    else:
        field_of_study.append(education[fos + 1])

    per_st_edu = educationDetails(education,"Dates attended or expected graduation", 0)
    if(per_st_edu == -9):
        period_attended.append("NO TIMELINE FOUND")
    else:
        period_attended.append(education[per_st_edu + 1])
    
#     proof_1 = br.find_element_by_id("education-section")
#     uno_proof = proof_1.find_elements_by_tag_name("li")[0].text.split("\n")[0]
#     gpa_person = educationDetails(education,"Grade", 0)
#     y = re.findall(r'.GPA |gpa ',education[gpa_person + 1])
#     if(len(y)>1):
#         education[gpa_person + 1] = float(x.split(y[0])[-1])
        
#     if(gpa_person == -9 and uno_proof == "Virginia Tech" or type(education[gpa_person + 1]) == type("Qasim Wani")):
#         gpa.append(0) 
#     else:
#         gpa.append(education[gpa_person + 1])
    
#     if(isElement(".activities-societies") == True):
#         xts = br.find_element_by_css_selector(".activities-societies")
#         av = xts.find_element_by_xpath("..")
#         avx = av.find_element_by_xpath("..")
#         proof = avx.text.split("\n")[0]
#         proof = proof.split("Virginia Tech")[0]
        
#         if(proof == "Virginia Tech"):
#             act_soc = br.find_element_by_css_selector(".activities-societies")
#             activities.append(act_soc.text)
#         else:
#             activities.append("No Activity")
#     else:
#         activities.append("No Activity")

In [ ]:
def userSkills():
    br.execute_script("window.scrollTo(0, 800);")
    q_skill = ".pv-profile-section__card-action-bar.pv-skills-section__additional-skills.artdeco-container-card-action-bar"
    if(isElement(q_skill) == True):
        br.find_element_by_css_selector(q_skill).click()

    list_of_user_skills = ".pv-skill-category-entity__name-text.t-16.t-black.t-bold"

    if(isElement(list_of_user_skills) == True):
        ser_skil = br.find_elements_by_css_selector(list_of_user_skills)
        one_skill = []
        for x in ser_skil:
            one_skill.append(x.text + " ")
        skills.append(one_skill)
    else:
        skills.append(['NO SKILLS LISTED'])

In [ ]:
def user_has_interests():
    xyz = ".pv-profile-section.pv-interests-section.artdeco-container-card.ember-view"
    if(isElement(xyz) == True):
        interests = br.find_element_by_css_selector(xyz)
        allInt = interests.text.split("\n")
        user_interests.append(allInt[1:-1:2])
    else:
        user_interests.append("No Interests Listed")

# The Main Function Invoking Everything Else

In [ ]:
time.sleep(1)
next_button = ".artdeco-pagination__button.artdeco-pagination__button--next.artdeco-button.artdeco-button--muted.artdeco-button--icon-right.artdeco-button--1.artdeco-button--tertiary.ember-view"
br.execute_script("window.open('https://www.google.com');")
for r in range(1):
    time.sleep(1.5)
    individualProfiles()
    time.sleep(0.5)
    for slow in range(30):
        br.execute_script("window.scrollTo(0,"+ str(slow*100) +");")
        time.sleep(0.3)
    br.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    br.find_element_by_css_selector(next_button).click()

In [ ]:
#Collection of all data
big_data =  list(zip(imageLinks,userName, biographyUser, schooling, degree_name, field_of_study, period_attended,skills, user_interests))
col = ['Image Links', 'Username','Biography','University Name','Degree Name','Major', 'Period Attended','Skills','Interests']
df = pd.DataFrame(data=big_data,columns=col)
backup = df

In [ ]:
# df.to_csv('dataScrapingLinkedIn.csv',index=False,encoding='utf-8')
# backup.drop_duplicates(subset ="Username", keep = False, inplace = True) 
# df
print(len(userName))

In [ ]:
time.sleep(10)
br.execute_script("window.open('https://evilens.herokuapp.com/login');")
time.sleep(10)
br.switch_to.window(br.window_handles[-1])

In [ ]:
pantomath_username = br.find_element_by_id("user_login_name")
pantomath_username.send_keys("hacker")
pantomath_password = br.find_element_by_id("user_login_password")
pantomath_password.send_keys("qasimwani")
pantomath_password.submit()

In [ ]:
all_user_data = []
for i in range(len(df)):
    all_user_data.append(df.loc[i])
    

In [ ]:
def publish(one,two,three,four,five,six,seven,eight,nine):
    scraped_user_link = br.find_element_by_id("scraped_user_link").send_keys(one)
    scraped_userName = br.find_element_by_id("scraped_user_name").send_keys(two)
    scraped_bio = br.find_element_by_id("user_scraped_biography").send_keys(three)
    scraped_uni_name = br.find_element_by_id("scraped_user_universityName").send_keys(four)
    scraped_degreeName = br.find_element_by_id("scraped_degree_name").send_keys(five)
    scraped_major = br.find_element_by_id("scraped_user_major").send_keys(six)
    scraped_period_attended = br.find_element_by_id("period_attended").send_keys(seven)
    scraped_skills = br.find_element_by_id("user_scraped_skills").send_keys(eight)
    scraped_interests = br.find_element_by_id("user_scraped_interests").send_keys(nine)
    br.find_element_by_css_selector(".btn.btn-lg.btn-primary.btn-block").submit()

In [ ]:
for x in range(len(all_user_data)):
    time.sleep(0.5)
    newUser = br.find_element_by_css_selector(".btn.btn-primary.text-center.data_Point")
    newUser.click()
    publish(all_user_data[x][0],all_user_data[x][1],all_user_data[x][2],all_user_data[x][3],all_user_data[x][4],all_user_data[x][5],all_user_data[x][6],all_user_data[x][7][:],all_user_data[x][8][:],)

In [ ]:
def deleteAllPosts():
    time.sleep(0.36)
    br.execute_script("window.scrollTo(0, 300);")
    br.find_elements_by_css_selector(".btn.btn-primary")[1].click()
    time.sleep(0.36)
    br.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    br.find_element_by_css_selector(".btn.btn-sm.btn-danger").click()

In [ ]:
# choice = input("Do you want to delete all posts? ")
# if(choice == "yes"):
#     for x in range(len(all_user_data)):
#         deleteAllPosts()


In [ ]:
"70th to be scanned now. Start base at 70. i.e. run program at 70."